In [1]:
%%bash
mkdir ./samtools_out

## these steps are needed to open bam file in IGV :
- IGV takes sorted indexed `bam` file as an input

#### 1. convert a file from `sam` into `bam` format : 

In [1]:
%%bash
samtools view \
-b -S -o \
./samtools_out/saratov_map_unpaired.bam \
./bowtie_out/saratov_map_unpaired.sam 

#### 2. sort reads by their genome positions :

In [2]:
%%bash
samtools sort ./samtools_out/saratov_map_unpaired.bam > ./samtools_out/saratov_map_unpaired_sorted.bam

#### 3. remove pcr duplicates from sorted file :

In [3]:
%%bash
samtools rmdup  ./samtools_out/saratov_map_unpaired_sorted.bam ./samtools_out/saratov_map_unpaired_sorted_rmdup.bam

[bam_rmdup_core] processing reference AF409138.1...


#### 4. indexing sorted file : 

In [4]:
%%bash
samtools index ./samtools_out/saratov_map_unpaired_sorted_rmdup.bam

#### now you can open file `saratov_map_unpaired_sorted_rmdup.bam` with IGV to visualize mapping reads and variants :_

![igv](./pictures/igv_example.png)

we can also import `sam` file into Ugene (it sorts, creates index and makes file `ugenedb`, which can be opened in Ugene afterwards) to visualize the mapped reads :

![ugene](./pictures/ugene_example.png)

In [3]:
%%bash
ls ./input_data/


1_trim_crop_and_trail_32_1P.fastq
1_trim_crop_and_trail_32_1U.fastq
1_trim_crop_and_trail_32_2P.fastq
1_trim_crop_and_trail_32_2U.fastq
2_trim_crop_and_trail_32_2P.fastq
SUD-Saratov_S1_L001_R1_001.fastq
SUD-Saratov_S1_L001_R2_001.fastq
input_data.rar
merged_document.aln
ref_AF409138.1.fasta
ref_AF409138.1.fasta.fai
saratov_map_on_AF_consensus.fa


####  6. calculating variants
- `mpileup` calculates likelihoods of the variants
- we can skip indels by `--skip-indels` flag (can be denoted as `-I` for short)
- `--BCF` is the output option (output in binary call format) and may be `-g` for short
- `--fasta` denots reference file format, can be `-f` for short

In [6]:
%%bash
samtools mpileup --BCF \
--fasta ./input_data/ref_AF409138.1.fasta \
./samtools_out/saratov_map_unpaired_sorted_rmdup.bam > ./samtools_out/saratov_vars.bcf

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


#### 7. calling variants. conventional variant
- `-c`, `--consensus-caller`  the original samtools/bcftools calling method (conflicts with -m)
- `-v`, `--variants-only` output variant sites only
- `-V`, `--skip-variants snps|indels` skip indel/SNP sites
- `--ploidy-file` path to the file which denotes set of chromosomes, if not set, all samples are considered diploid. It's better to set it to improve variants call quality (use to get help : `bcftools call --ploidy ?`)

In [6]:
%%bash
bcftools call \
--consensus-caller \
--variants-only \
--ploidy-file ./samtools_out/ploid.txt \
./samtools_out/saratov_vars.bcf > ./samtools_out/saratov_vars.vcf

#### from here we can parse `vcf` to explore the difference from the reference

In [13]:
import vcf

In [12]:
vcf_reader = vcf.Reader(open('./samtools_out/saratov_vars.vcf', 'r'))
counter = 0
for record in vcf_reader:
    print(record)
    print(record.INFO["DP"])
    print(record.INFO["DP4"])
    print(record.INFO)
    print("--------------------")
    counter += 1

print(counter)
    

Record(CHROM=AF409138.1, POS=502, REF=A, ALT=[G])
47
[0, 0, 14, 26]
{'DP': 47, 'VDB': 0.242997, 'SGB': -0.693145, 'MQSB': 0.20277, 'MQ0F': 0.340426, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 14, 26], 'MQ': 1, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=759, REF=C, ALT=[T])
24
[0, 0, 7, 15]
{'DP': 24, 'VDB': 0.001429, 'SGB': -0.692562, 'MQSB': 0.760398, 'MQ0F': 0.166667, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 7, 15], 'MQ': 1, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=5386, REF=A, ALT=[G])
10
[0, 0, 1, 7]
{'DP': 10, 'VDB': 0.020653, 'SGB': -0.651104, 'MQSB': 1.0, 'MQ0F': 0.1, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 1, 7], 'MQ': 21, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=5498, REF=T, ALT=[G])
21
[0, 0, 13, 6]
{'DP': 21, 'VDB': 0.650357, 'SGB': -0.69168, 'MQSB': 0.850016, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 13, 6], 'MQ': 39, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=5534, REF=A, ALT=[G])
16
[0,

[0, 0, 21, 3]
{'DP': 28, 'VDB': 1.35148e-05, 'SGB': -0.692831, 'MQSB': 0.271496, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 21, 3], 'MQ': 34, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=121712, REF=A, ALT=[G])
2
[0, 0, 2, 0]
{'DP': 2, 'VDB': 0.32, 'SGB': -0.453602, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 2, 0], 'MQ': 41, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=121814, REF=A, ALT=[G])
25
[0, 0, 11, 10]
{'DP': 25, 'VDB': 0.00705183, 'SGB': -0.692352, 'MQSB': 0.447846, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 11, 10], 'MQ': 37, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=121839, REF=GAA, ALT=[G])
8
[0, 0, 6, 2]
{'INDEL': True, 'IDV': 5, 'IMF': 0.625, 'DP': 8, 'VDB': 0.00319992, 'SGB': -0.651104, 'MQSB': 0.25, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 6, 2], 'MQ': 18, 'FQ': -999.0}
--------------------
Record(CHROM=AF409138.1, POS=121840, REF=AAT, ALT=[A])
4
[0, 0, 3, 0]
{'INDEL': True, 'I

#### total number of SNPs in vcf :

In [14]:
vcf_reader = vcf.Reader(open('./samtools_out/saratov_vars.vcf', 'r'))
counter = 0
for record in vcf_reader:
    if "INDEL" not in record.INFO.keys():
        counter += 1

print("total SNPs : ", counter)

total SNPs :  520



####  DP4 contains 4 values:
1. forward ref alleles
2. reverse ref
3. forward non-ref 
4. reverse non-ref alleles

Sum can be smaller than DP (The number of reads covering or bridging POS)  because low-quality bases are not counted.

#### SNPs with coverage more than 10:

In [15]:
vcf_reader = vcf.Reader(open('./samtools_out/saratov_vars.vcf', 'r'))
counter = 0
for record in vcf_reader:
    if "INDEL" not in record.INFO.keys():
        if record.INFO["DP4"][2] + record.INFO["DP4"][3] >= 10:
            counter += 1
print("SNP passing filter: ", counter)

SNP passing filter:  388


#### 8. consensus calling
- to create consensus we apply variants to the reference

#### NOTE : if we want to get consensus, we'll have to redo the calling step, as bcftools can apply only files compressed with bgzip to the reference. If we want just to get vcf file, we don't need this step 

the command `-Oz -o` means ouput in bzip format 
- we need this compressed format to get consensus (otherwise it doesn't work)
- then we must index compressed `vcf` file, we need it derive consensus as well

In [8]:
%%bash
bcftools call \
--consensus-caller \
--variants-only \
--skip-variants indels \
--ploidy-file ./samtools_out/ploid.txt \
./samtools_out/saratov_vars.bcf \
-Oz -o ./samtools_out/saratov_vars.vcf.gz \

#### we must index the `lsd_2_vars.vcf.gz`

In [9]:
%%bash
bcftools tabix -p vcf ./samtools_out/saratov_vars.vcf.gz

#### creating consensus

Create consensus sequence by applying VCF variants to a reference fasta file.

- reference sequence
- `bcf` file with variants calls
- indexed `vcf` file

but:
- we didn't call indels, so we don't need to normalize and filter indels

see : http://samtools.github.io/bcftools/howtos/consensus-sequence.html

In [5]:
%%bash
ls -l ./input_data

total 446644
-rwxrwxrwx 1 root root  33407745 Feb 15 10:45 1_trim_crop_and_trail_32_1P.fastq
-rwxrwxrwx 1 root root   1921926 Feb 15 10:45 1_trim_crop_and_trail_32_1U.fastq
-rwxrwxrwx 1 root root  33002253 Feb 15 10:45 1_trim_crop_and_trail_32_2P.fastq
-rwxrwxrwx 1 root root    186221 Feb 15 10:45 1_trim_crop_and_trail_32_2U.fastq
-rwxrwxrwx 1 root root  26364751 Feb 18 10:44 2_trim_crop_and_trail_32_2P.fastq
-rwxrwxrwx 1 root root 133352346 Jun 25  2018 SUD-Saratov_S1_L001_R1_001.fastq
-rwxrwxrwx 1 root root 138569286 Jun 25  2018 SUD-Saratov_S1_L001_R2_001.fastq
-rwxrwxrwx 1 root root  88870486 Feb 18 12:59 input_data.rar
-rwxrwxrwx 1 root root   1363903 Feb 14 16:47 merged_document.aln
-rwxrwxrwx 1 root root    152749 Jun 25  2018 ref_AF409138.1.fasta
-rwxrwxrwx 1 root root        27 Feb 14 17:40 ref_AF409138.1.fasta.fai
-rwxrwxrwx 1 root root    150532 Feb 14 16:43 saratov_map_on_AF_consensus.fa


In [10]:
%%bash
cat ./input_data/ref_AF409138.1.fasta | \
bcftools consensus \
./samtools_out/saratov_vars.vcf.gz \
> ./samtools_out/consensus_seq_.fasta

Note: the --sample option not given, applying all records
